# SRS CSI Interactive Analysis

This notebook provides interactive visualization tools for analyzing SRS (Sounding Reference Signal) Channel State Information collected from the gNB.

**Configuration:**
- Band 78 TDD, 20 MHz BW
- Subcarrier Spacing: 30 kHz
- Symbol Duration: ~71.35 μs
- SRS Period: 40 ms

**Features:**
- Frequency response visualization per SRS occasion
- Time evolution of channel at specific subcarriers
- Interactive widgets for exploration

## 1. Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import interact

plt.style.use('ggplot')
%matplotlib inline

print("✓ Libraries imported successfully")

## 2. Configure Data Path and Symbol Timing

In [ ]:
# Path to the SRS CSI CSV file
data_path = Path('srs_csi_20260108_204150_1.csv')

# Symbol timing for 30 kHz SCS (your configuration)
# 0.5 ms slot / 14 symbols = 71.35 μs per symbol
SYMBOL_DURATION_US = 71.35

print(f"Data file: {data_path}")
print(f"Symbol duration: {SYMBOL_DURATION_US:.2f} μs (30 kHz SCS)")

## 3. Load and Preprocess SRS CSI Data

In [ ]:
if not data_path.exists():
    raise FileNotFoundError(f"Could not locate {data_path}")

print(f"Loading {data_path}...")
df = pd.read_csv(data_path)

# Calculate complex channel response H(k)
df['H'] = df['real'].astype(float) + 1j * df['imag'].astype(float)
df['absH'] = np.abs(df['H'])
df['phase_rad'] = np.angle(df['H'])
df['phase_deg'] = np.angle(df['H'], deg=True)

print(f"✓ Loaded {len(df):,} CSI samples")

## 4. Compute Symbol Timestamps

In [ ]:
# Calculate per-symbol timestamps
if 'symbol' in df.columns:
    df['symbol_time_us'] = df['timestamp_us'].astype(float) + df['symbol'].astype(float) * SYMBOL_DURATION_US
else:
    df['symbol_time_us'] = df['timestamp_us'].astype(float)

# Convert to milliseconds for easier interpretation
df['symbol_time_ms'] = (df['symbol_time_us'] - df['symbol_time_us'].min()) / 1e3

print(f"✓ Symbol timestamps computed")

## 5. Display Dataset Summary Statistics

In [ ]:
capture_duration_ms = (df['timestamp_us'].max() - df['timestamp_us'].min()) / 1e3

print("="*70)
print("SRS CSI DATASET SUMMARY")
print("="*70)
print(f"Total CSI samples:    {len(df):,}")
print(f"SRS occasions:        {df['entry_num'].nunique():,}")
print(f"Capture duration:     {capture_duration_ms:.2f} ms")
print(f"RX antenna ports:     {sorted(df['rx_port'].unique())}")
print(f"TX antenna ports:     {sorted(df['tx_port'].unique())}")
print()

print("Resource Configuration:")
display(df.groupby('num_tones').size().to_frame('SRS_occasions'))

print("\nSymbol Distribution:")
display(df['symbol'].value_counts().sort_index().to_frame('count'))

print(f"\nSubcarrier Range:")
print(f"  Min: {df['subcarrier'].min()}")
print(f"  Max: {df['subcarrier'].max()}")
print(f"  Unique subcarriers: {df['subcarrier'].nunique()}")
if len(df) > 1:
    comb_spacing = df['subcarrier'].diff().mode()[0]
    print(f"  SRS comb spacing: {comb_spacing:.0f} subcarriers")

print(f"\nChannel Magnitude Statistics:")
print(f"  Mean |H(k)|: {df['absH'].mean():.4f}")
print(f"  Std |H(k)|:  {df['absH'].std():.4f}")
print(f"  Min |H(k)|:  {df['absH'].min():.4f}")
print(f"  Max |H(k)|:  {df['absH'].max():.4f}")

## 6. Interactive Visualization: Frequency Response per SRS Occasion

Use the sliders below to explore the frequency response (magnitude and phase across subcarriers) for different SRS occasions.

In [ ]:
# Get valid ranges for interactive widgets
valid_rx_ports = sorted(df['rx_port'].unique())
entry_min = int(df['entry_num'].min())
entry_max = int(df['entry_num'].max())
subcarrier_min = int(df['subcarrier'].min())
subcarrier_max = int(df['subcarrier'].max())
default_subcarrier = int(df['subcarrier'].median())

def plot_entry_vs_subcarrier(rx_port=valid_rx_ports[0], entry_num=entry_min):
    """Plot frequency response for a single SRS occasion"""
    data = df[(df['rx_port'] == rx_port) & (df['entry_num'] == entry_num)].sort_values('subcarrier')
    
    if data.empty:
        print(f'No CSI samples for RX port {rx_port}, entry {entry_num}.')
        return
    
    fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)
    
    # Magnitude plot
    axes[0].plot(data['subcarrier'], data['absH'], '.-', markersize=4)
    axes[0].set_ylabel('|H(k)|', fontsize=12)
    axes[0].set_title(f'SRS Entry {entry_num}, RX Port {rx_port}: Frequency Response', fontsize=14)
    axes[0].grid(True, alpha=0.3)
    
    # Phase plot
    axes[1].plot(data['subcarrier'], data['phase_deg'], '.-', markersize=4, color='orange')
    axes[1].set_ylabel('Phase (degrees)', fontsize=12)
    axes[1].set_xlabel('Subcarrier Index', fontsize=12)
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Create interactive widget
interact(
    plot_entry_vs_subcarrier,
    rx_port=widgets.Dropdown(options=valid_rx_ports, value=valid_rx_ports[0], description='RX Port'),
    entry_num=widgets.IntSlider(min=entry_min, max=entry_max, value=entry_min, step=1,
                                description='Entry #', continuous_update=False),
)

## 7. Interactive Visualization: Time Evolution at Specific Subcarrier

Explore how the channel response changes over time at a specific subcarrier (useful for Doppler analysis).

In [ ]:
def plot_subcarrier_vs_time(rx_port=valid_rx_ports[0], subcarrier=default_subcarrier):
    """Plot time evolution of CSI at a specific subcarrier"""
    data = df[(df['rx_port'] == rx_port) & (df['subcarrier'] == subcarrier)].sort_values('symbol_time_us')
    
    if data.empty:
        print(f'No CSI samples for RX port {rx_port}, subcarrier {subcarrier}.')
        return
    
    # Calculate time offset in milliseconds
    time_offset_ms = (data['symbol_time_us'] - data['symbol_time_us'].iloc[0]) / 1e3
    
    fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)
    
    # Magnitude over time
    axes[0].plot(time_offset_ms, data['absH'], '.-', markersize=3)
    axes[0].set_ylabel('|H(k)|', fontsize=12)
    axes[0].set_title(f'Subcarrier {subcarrier}, RX Port {rx_port}: Time Evolution', fontsize=14)
    axes[0].grid(True, alpha=0.3)
    
    # Phase over time
    axes[1].plot(time_offset_ms, data['phase_deg'], '.-', markersize=3, color='orange')
    axes[1].set_ylabel('Phase (degrees)', fontsize=12)
    axes[1].set_xlabel('Time offset (ms)', fontsize=12)
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Create interactive widget
interact(
    plot_subcarrier_vs_time,
    rx_port=widgets.Dropdown(options=valid_rx_ports, value=valid_rx_ports[0], description='RX Port'),
    subcarrier=widgets.IntSlider(min=subcarrier_min, max=subcarrier_max, value=default_subcarrier, step=4,
                                 description='Subcarrier', continuous_update=False),
)